In [1]:
from google.colab import drive
import keras
from keras import Sequential
from keras.layers import Dense, Activation, Dropout, BatchNormalization
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler,LabelEncoder

sns.set()

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
drive.mount('/gdrive')

In [0]:
pd.options.display.max_rows=15

# **데이터 전처리**

In [0]:
train = pd.read_csv('/gdrive/My Drive/Titanic/train.csv')
test = pd.read_csv('/gdrive/My Drive/Titanic/test.csv')

train_test_data = [train, test]

In [0]:
for dataset in train_test_data:
    dataset.loc[dataset['Sex']=='male','Sex'] = 0
    dataset.loc[dataset['Sex']=='female','Sex'] = 1

In [0]:
for dataset in train_test_data:
  Ticket_data = dataset["Ticket"].value_counts().sort_index()
  Ticket_list = Ticket_data.index.tolist()
  print(Ticket_list)
  dataset["Ticket_Group"] = 0
  for idx in range(len(dataset)):
    dataset["Ticket_Group"][idx] = Ticket_list.index(dataset["Ticket"][idx])

In [0]:
for dataset in train_test_data:
  dataset['Embarked'] = dataset['Embarked'].fillna('S')
  dataset['Embarked'] = dataset['Embarked'].astype(str)

In [0]:
for dataset in train_test_data:
  dataset['Fare'] = dataset['Fare'].fillna(7.180641176470588)

In [0]:
for dataset in train_test_data:
  dataset.loc[dataset['Age'].isnull()==True,'Age_Nan'] = 1
  dataset.loc[dataset['Age'].isnull()==False,'Age_Nan'] = 0 

In [0]:
for dataset in train_test_data:
  dataset.loc[dataset['Cabin'].isnull()==True,'Cabin_Nan'] = 1
  dataset.loc[dataset['Cabin'].isnull()==False,'Cabin_Nan'] = 0 

In [0]:
for dataset in train_test_data:
  dataset['Fname'] = dataset['Name'].map(lambda x: x.split(',')[1].split('.')[0].strip())

In [0]:
train.loc[796,"Fname"] = "Mrs"

In [0]:
train['Fname'].replace(['Capt','Col','Don','Dr','Jonkheer','Lady','Major','Master','Miss','Mlle','Mme','Mr','Mrs','Ms','Rev','Sir','the Countess'],
                      ['Mr','Mr','Mr','Mr','Mr','Mrs','Mr','Master','Miss','Miss','Miss','Mr','Mrs','Miss','Mr','Mr','Mrs'],inplace=True)

In [0]:
test['Fname'].replace(['Col','Dona','Dr','Master','Miss','Mr','Mrs','Ms','Rev'],
                      ['Mr','Mrs','Mr','Master','Miss','Mr','Mrs','Miss','Mr'],inplace=True)

In [0]:
for dataset in train_test_data:
  dataset.loc[dataset['Fname']=='Mr','Fname'] = 0
  dataset.loc[dataset['Fname']=='Master','Fname'] = 1
  dataset.loc[dataset['Fname']=='Miss','Fname'] = 2
  dataset.loc[dataset['Fname']=='Mrs','Fname'] = 3

# **정규화**

In [0]:
for dataset in train_test_data:
    dataset['Fname'] = StandardScaler().fit_transform(dataset['Fname'].values.reshape(-1, 1))
    dataset['Cabin_Nan'] = StandardScaler().fit_transform(dataset['Cabin_Nan'].values.reshape(-1, 1))
    dataset['Age_Nan'] = StandardScaler().fit_transform(dataset['Age_Nan'].values.reshape(-1, 1))
    dataset['Fare'] = StandardScaler().fit_transform(dataset['Fare'].values.reshape(-1, 1))
    dataset['Pclass'] = StandardScaler().fit_transform(dataset['Pclass'].values.reshape(-1, 1))
    dataset['Sex'] = StandardScaler().fit_transform(dataset['Sex'].values.reshape(-1, 1))
    dataset['SibSp'] = StandardScaler().fit_transform(dataset['SibSp'].values.reshape(-1, 1))
    dataset['Parch'] = StandardScaler().fit_transform(dataset['Parch'].values.reshape(-1, 1))
    dataset['Ticket_Group'] = StandardScaler().fit_transform(dataset['Ticket_Group'].values.reshape(-1, 1))

# **불필요한 열 삭제**

In [0]:
train.drop(['Cabin', 'Age', 'Embarked', 'Ticket', 'Name', 'PassengerId'], axis=1, inplace=True)
test.drop(['Cabin', 'Age', 'Embarked', 'Ticket', 'Name'], axis=1, inplace=True)

In [0]:
target = train['Survived']
train_data = train.drop('Survived', axis=1)

In [0]:
print(test.shape)
print(target.shape)
print(train_data.shape)

# **TEST**

In [0]:
k_fold = KFold(n_splits=10, shuffle=True, random_state=0)
clf = SVC()
# clf = DecisionTreeClassifier()
clf.fit(train_data, target)
 
test_data = test.drop("PassengerId", axis=1).copy()
prediction = clf.predict(test_data)
submission = pd.DataFrame({
        "PassengerId": test["PassengerId"],
        "Survived": prediction
    })
 
submission.to_csv('/gdrive/My Drive/Titanic/submission_Fname.csv', index=False)
submission = pd.read_csv('/gdrive/My Drive/Titanic/submission_Fname.csv')
submission.head()